In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import mediapipe as mp 
import cv2
from numpy import random
import pickle
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
with open('rps_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Loading images
    rock_img = cv2.imread('rock.jpg')
    paper_img = cv2.imread('paper.jpg')
    scissors_img = cv2.imread('scissors.jpg')
    
    actions = {"Rock":0, "Paper":1, "Scissors":2}
    actions_list = ["Rock", "Paper", "Scissors"]
    raw_imgs = [rock_img, paper_img, scissors_img]
    icons = []
    for img in raw_imgs:
        icons.append(cv2.resize(img, (200, 200), interpolation = cv2.INTER_LINEAR))

    # Setting game variables
    prev_move = 3
    curr_move = 3
    comp_move = 3
    winner = 2

    game_score = [0,0,0]
    
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = holistic.process(image)

        # face, pose, left hand, right hand
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Naming a window 
        cv2.namedWindow("Rock Paper Scissors", cv2.WINDOW_NORMAL) 
          
        # Using resizeWindow() 
        cv2.resizeWindow("Rock Paper Scissors", 960, 720) 
        
        # right hand
        #mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Adding visuals
        cv2.rectangle(image, (390,150), (590, 350), (247, 117, 16), 5)
        cv2.rectangle(image, (390,120), (550, 150), (247, 117, 16), -1)
        cv2.putText(image, 'Computer Move', 
                       (400, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

        # Export Coordinates
        try:
            
            # Extract hand landmarks
            hand = results.right_hand_landmarks.landmark
            hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in hand]).flatten())
            
            #row = pose_row+face_row
            row = hand_row

            # Make detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)

            # Updating game variables
            curr_move = actions[body_language_class]
            if (curr_move != prev_move) or (curr_move == 3):
                prev_move = curr_move
                comp_move = random.randint(3)

                if (curr_move == 0 and comp_move == 2) or \
                   (curr_move == 1 and comp_move == 0) or \
                   (curr_move == 2 and comp_move == 1):
                    winner = 0
                elif curr_move == comp_move:
                    winner = 2
                else:
                    winner = 1
                game_score[winner] += 1

            # Playing game
            image[ 150:350 , 390:590 ] = icons[comp_move]
            cv2.putText(image, actions_list[comp_move], 
                   (445,330), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (247, 117, 16), -1)
            # Display class
            cv2.putText(image, 'Player Move', 
                       (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0], 
                       (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

            # Display probability
            cv2.putText(image, 'PROB', 
                       (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2)), 
                       (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        except:
            pass

        # Display game score
        cv2.rectangle(image, (190,400), (390, 450), (247, 117, 16), -1)
        cv2.putText(image, 'Computer', 
                       (285, 412), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(game_score[1]), 
                   (280,440), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        # Display probability
        cv2.putText(image, 'Player', 
                   (200, 412), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(game_score[0]), 
                   (195,440), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        cv2.imshow("Rock Paper Scissors", image)
    
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
cap.release()
cv2.destroyAllWindows()